In [ ]:
import pandas as pd
import geopandas as gpd
import sqlalchemy
from geoalchemy2 import Geometry, WKTElement
import src.transform as t





In [ ]:
engine = pass

trips_long = gpd.read_postgis('trips_long', engine, geom_col='geom_coord')
trips_long

In [ ]:
locations_raw = clean.plus_minus_locations(rides_long, 'ANC')
locations_raw

In [ ]:
%%time

three_hour_rolling = clean.cumulative_change(locations_raw, window_size='3H')
three_hour_rolling

In [ ]:
# Which series have the largest ranges? About half of these perform much differently than the others
# Worth noting that ANC's in seven and eight have very light differencing
min_max_diff = three_hour_rolling.aggregate(['min','max']).transpose()
min_max_diff.columns = ['minimum', 'maximum']
min_max_diff['range'] = min_max_diff.maximum - min_max_diff.minimum
min_max_diff.sort_values('range', ascending=False)

## There are large spikes here around for example early June, but let's see how SARIMA does, with a daily seasonal component (24 hours per day)

In [ ]:
viz.plot_series(three_hour_rolling['5D'],train_end='2020-07-01')

In [ ]:
series = three_hour_rolling['5D'].resample('1H').mean()
viz.plot_series(series,train_end='2020-07-01')

In [ ]:
series = three_hour_rolling['1A'].resample('1H').mean()

# Leaving aside month of July as a Test Set, this is about 25% of the data
test_start = '2020-07-01 00:00:00'

# Make the train/test splits
train = series[series.index < test_start]
test = series[test_start:]

# Checking output lens add up to len of series (they do)
print(len(series), len(test), len(train))

# model
model = SARIMAX(train, order=(0,1,1), seasonal_order=(0,1,1,24))

In [ ]:
# Fit Model
result = model.fit()

In [ ]:
preds = result.predict(start=train.index[0], end=test.index[-1], dynamic=test_start)

## SARIMAX crushes it in-sample, even catching huge spikes, it does much less well as distance out of sample decreases

- Since our business use case is for fleet rebalancing on an intraday basis rather than predicting out of sample forecast one month out, it seems quite fair to use walk-forward modeling here
- Future improvement: would reaaaaallly like to add a seasonal weekly component with s=168 (i.e. both daily s=24, and weekly s=168)

In [ ]:
fig, ax = viz.plot_series(series, train_end=test_start, preds=preds, fig_size=(15,15))

In [ ]:
ax.set_title('Naive SARIMA model Prediction 1 Month out of Sample for ANC 1A \n config=(0,1,1)(0,1,1,24)')
fig.savefig('../figures/declining_out_of_sample.png')

In [ ]:
ax.set_xlim(ax.set_xlim([dt.date(2020, 6, 30), dt.date(2020, 7, 2)]))
ax.set_title('Naive SARIMA model Prediction 1 Day out of Sample for ANC 1A \n config=(0,1,1)(0,1,1,24) -- Full Train Set Starts 2020-04-01')
fig.savefig('../figures/one_day_pred_no_walk_forward.png')
fig

In [ ]:
summary_daily1A = result.summary()
summary_daily1A

## Attempting a Weekly Seasonal with s=168

In [ ]:
series = three_hour_rolling['1A'].resample('1H').mean()

# Leaving aside month of July as a Test Set, this is about 25% of the data
test_start = '2020-07-01 00:00:00'

# Make the train/test splits
train = series[series.index < test_start]
test = series[test_start:]

# Checking output lens add up to len of series (they do)
print(len(series), len(test), len(train))

# model
model = SARIMAX(train, order=(0,1,1), seasonal_order=(0,1,1,168))

In [ ]:
result = model.fit()

In [ ]:
preds = result.predict(start=train.index[0], end=test.index[-1], dynamic=test_start)

In [ ]:
recent_geo.started_at.value_counts().head(60)

In [ ]:
cleaned_recent[cleaned_recent.started_at > cleaned_recent.ended_at]

In [ ]:
recent_geo[recent_geo.started_at.isin(recent_geo.started_at.value_counts().head(60).index) & (recent_geo.duration_seconds == 1118.0)].start_station_name.value_counts()

In [ ]:
recent_geo[recent_geo.started_at == '2020-07-23 11:24:54']

In [ ]:
historical = pd.concat(raw_dfs[:-4])
historical